## Importando as bibliotecas e pacotes

In [28]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
import numpy as np

## Carregando o dataset

In [ ]:
path = "https://raw.githubusercontent.com/matheusssilva991/hackaton-cepedi/main/data/dados_consumo_agua.csv"
df_dados_agua = pd.read_csv(path, sep = ",")

## Breve resumo do dataset

In [ ]:
df_dados_agua.head()

In [ ]:
df_dados_agua.dtypes

In [ ]:
df_dados_agua.info()

## Modificando tipos dos dados do dataset

### Diminuindo o peso dos dados

In [ ]:
df_dados_agua["ano"] = df_dados_agua["ano"].astype("uint16")
df_dados_agua["mes"] = df_dados_agua["mes"].astype("uint8")

### Modificando a coluna *"dia_semana"* para manter os dados de forma numérica

#### Verificando os valores únicos

In [ ]:
df_dados_agua["dia_semana"].unique()

#### Mapeando os novos valores de "dia_semana"

In [ ]:
dicionario_dia_semana = {
    "domingo"   : 0,
    "segunda"   : 1,
    "terça"     : 2,
    "quarta"    : 3,
    "quinta"    : 4,
    "sexta"     : 5,
    "sábado"    : 6
}

df_dados_agua["dia_semana"] = df_dados_agua["dia_semana"].map(dicionario_dia_semana)
df_dados_agua["dia_semana"] = df_dados_agua["dia_semana"].astype("uint8")

### Modificando as outras colunas do dataset

In [ ]:
# dropando coluna "dia_mes" por redundancia
df_dados_agua.drop(columns = ["dia_mes"], axis = "columns", inplace = True)

df_dados_agua["hora"] = df_dados_agua["hora"].astype("uint8")
df_dados_agua["quantidade_pessoas"] = df_dados_agua["quantidade_pessoas"].astype("uint8")
df_dados_agua["consumo_agua_m3"] = df_dados_agua["consumo_agua_m3"].astype("float32")

### Modificando as colunas *"regiao"* e *"padrao_consumo"* para manter os dados de forma numérica

#### Verificando os valores únicos

In [ ]:
print(f"Valores únicos da coluna ---regiao--- : {df_dados_agua.regiao.unique()} \n")
print(f"Valores únicos da coluna ---padrao_consumo--- : {df_dados_agua.padrao_consumo.unique()}")

#### Mapeando os novos valores de *"padrao_consumo"* e excluindo a coluna *"regiao"*

In [ ]:
# dropando coluna "regiao" por possuir um unico valor
df_dados_agua.drop(columns = ["regiao"], axis = "columns", inplace = True)

dicionario_padrao_consumo = {
    "Normal" : 0,
    "Alto" : 1
}

df_dados_agua["padrao_consumo"] = df_dados_agua["padrao_consumo"].map(dicionario_padrao_consumo)
df_dados_agua["padrao_consumo"] = df_dados_agua["padrao_consumo"].astype("uint8")

### Como ficaram os dados:

In [ ]:
df_dados_agua.info()

## EDA

### Valor máximo, Valor mínimo, Média, Mediana, Desvio padrão

In [ ]:
print("Valores máximos:\n")
for column in df_dados_agua.columns:
    print(f"Coluna {column}: {df_dados_agua[column].max()}")
    
print("\n\nValores mínimos:\n")
for column in df_dados_agua.columns:
    print(f"Coluna {column}: {df_dados_agua[column].min()}")
    
print("\n\nMédia dos valores:\n")
for column in df_dados_agua.columns:
    print(f"Coluna {column}: {df_dados_agua[column].mean()}")
    
print("\n\nMediana dos valores:\n")
for column in df_dados_agua.columns:
    print(f"Coluna {column}: {df_dados_agua[column].median()}")
    
print("\n\nDesvio padrão dos valores:\n")
for column in df_dados_agua.columns:
    print(f"Coluna {column}: {df_dados_agua[column].std()}")

### Correlação entre as colunas

In [ ]:
df_correlacao = df_dados_agua.corr()
df_correlacao

### Matriz de correlação - Heatmap

In [ ]:
figure = plt.figure(figsize = (14, 12))
sns.heatmap(data = df_correlacao)
plt.show()

## Separando as features do target

In [29]:
scaler = MinMaxScaler()
scaler.fit(df_dados_agua.drop(columns = ["quantidade_pessoas", "padrao_consumo"]))
scaler.transform(df_dados_agua.drop(columns = ["quantidade_pessoas", "padrao_consumo"]))

array([[0.        , 0.        , 0.5       , 0.        , 0.0245098 ],
       [0.        , 0.        , 0.5       , 0.04347826, 0.04411764],
       [0.        , 0.        , 0.5       , 0.08695652, 0.05637255],
       ...,
       [1.        , 1.        , 1.        , 0.9130435 , 0.03186274],
       [1.        , 1.        , 1.        , 0.95652175, 0.02205882],
       [1.        , 1.        , 1.        , 1.        , 0.04411764]],
      dtype=float32)

In [ ]:
X = df_dados_agua.drop(columns = ["quantidade_pessoas", "padrao_consumo"]).values
y = df_dados_agua["padrao_consumo"].values.reshape(-1, 1)

## Divisão treino-teste

In [ ]:
tamanho_corte = 0.4
semente = 42
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = tamanho_corte, random_state = semente)

## RandomForestClassifier

### Criando o modelo RandomForest

In [ ]:
modelo_random_forest = RandomForestClassifier()

### Parametros para o RandomForest

In [ ]:
dicionario_random_forest = {
    "n_estimators"      : np.arange(10, 110, 10),# de 10 a 100
    "criterion"         : ["gini", "entropy", "log_loss"],
    "max_depth"         : np.arange(2, 8, 1),# de 2 a 7
    "min_samples_split" : np.arange(2, 11, 1),# de 2 a 10
    "min_samples_leaf"  : np.arange(1, 6, 1),# de 1 a 5
    "max_features"      : ["sqrt", "log2", None],
    "max_leaf_nodes"    : [5, 10, 15, 20, None],
    "n_jobs"            : [-1],
    "random_state"      : [semente]
}

### Utilizando o GridSearchCV

In [ ]:
grid_search_randomForest = GridSearchCV(
    estimator = modelo_random_forest,
    param_grid = dicionario_random_forest,
    cv = 5
)

### Treinando o GridSearch para obter os melhores parâmetros

In [ ]:
grid_search_randomForest.fit(x_train, y_train)
grid_search_randomForest.best_params_

### Imprimindo o Score do melhor modelo

In [ ]:
grid_search_randomForest.score(x_train, y_train)

## KNN

### Criando o modelo KNN

In [ ]:
modelo_knn = NearestNeighbors()

### Parametros para o KNN

In [ ]:
dicionario_knn = {
    "n_neighbors" : np.arange(3, 17, 2),# de 3 a 15
    "radius"      : np.arange(1.0, 3.5, 0.5),# de 1.0 a 3.0
    "p"           : np.arange(1.0, 2.1, 0.1),# de 1.0 a 2.0
    "n_jobs"      : [-1]
}

### Utilizando o GridSearchCV

In [ ]:
grid_search_KNN = GridSearchCV(
    estimator = modelo_knn,
    param_grid = dicionario_knn,
    cv = 5
)

### Treinando o GridSearch para obter os melhores parâmetros

In [ ]:
grid_search_KNN.fit(x_train, y_train)
grid_search_KNN.best_params_

### Imprimindo o Score do melhor modelo

In [ ]:
grid_search_KNN.score(x_train, y_train)

## SVC

### Criando o modelo SVM

In [ ]:
modelo_svc = SVC()

### Parametros para o SVM

In [ ]:
dicionario_svc = {
    "C"                         : np.logspace(-3, 3, 7),
    "kernel"                    : ["linear", "poly", "rbf", "sigmoid", "precomputed"],
    "degree"                    : np.arange(1, 6, 1),
    "tol"                       : np.logspace(-5, -1, 5),
    "max_iter"                  : [100, 200, 300, 400, 500, 600, -1],
    "decision_function_shape"   : ["ovr", "ovo"],
    "random_state"              :  [semente]
}

### Utilizando o GridSearchCV

In [ ]:
grid_search_svc = GridSearchCV(
    estimator = modelo_svc,
    param_grid = dicionario_svc,
    cv = 5
)

### Treinando o GridSearch para obter os melhores parâmetros

In [23]:
grid_search_svc.fit(x_train, y_train)
grid_search_svc.best_params_

/home/igorsssantana/anaconda3/envs/.cepedi-venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/igorsssantana/anaconda3/envs/.cepedi-venv/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/igorsssantana/anaconda3/envs/.cepedi-venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/igorsssantana/anaconda3/envs/.cepedi-venv/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (

KeyboardInterrupt: 

### Imprimindo o Score do melhor modelo

In [ ]:
grid_search_svc.score(x_train, y_train)